In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True' # very important

In [19]:
from PIL import Image, ImageDraw, ImageFont

import random
import numpy as np
import cv2

def generate_synthetic_dataset(output_dir, chars="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ", 
                               img_size=(32, 32), num_samples=50, font_paths=None):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load fonts
    if font_paths is None:
        font_paths = ["arial.ttf"]  # Replace with paths to license plate-like fonts
    fonts = [ImageFont.truetype(font, size=img_size[0] - 8) for font in font_paths]

    # Generate images
    for char in chars:
        char_dir = os.path.join(output_dir, char)
        os.makedirs(char_dir, exist_ok=True)
        
        for i in range(num_samples):
            # Create a blank image
            img = Image.new('L', img_size, color=255)  # White background
            draw = ImageDraw.Draw(img)

            # Choose random font and calculate bounding box
            font = random.choice(fonts)
            bbox = font.getbbox(char)
            w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]

            # Draw character in the center
            draw.text(((img_size[0] - w) / 2, (img_size[1] - h) / 2), char, fill=0, font=font)

            # Apply augmentations
            img = np.array(img)
            img = apply_augmentations(img)

            # Save image
            output_path = os.path.join(char_dir, f"{char}_{i}.png")
            cv2.imwrite(output_path, img)

    print(f"Synthetic dataset generated in '{output_dir}'")

def apply_augmentations(image):
    # Add random rotation
    angle = random.uniform(-15, 15)
    matrix = cv2.getRotationMatrix2D((image.shape[1] / 2, image.shape[0] / 2), angle, 1)
    image = cv2.warpAffine(image, matrix, (image.shape[1], image.shape[0]), borderValue=255)

    # Add random noise
    noise = np.random.normal(0, 10, image.shape).astype(np.uint8)
    image = cv2.add(image, noise)

    # Add random brightness variation
    brightness = random.uniform(0.7, 1.3)
    image = np.clip(image * brightness, 0, 255).astype(np.uint8)

    return image

# Example usage
generate_synthetic_dataset(
    output_dir="C:\\Users\\Nameet\\Downloads\\Synthetic Dataset",
    num_samples=100,
    font_paths=["arial.ttf", "times.ttf"]  # Replace with actual paths to font files
)


Synthetic dataset generated in 'C:\Users\Nameet\Downloads\Synthetic Dataset'
